In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from lightgbm.sklearn import LGBMRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score

import sklearn
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve,auc
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score


## Import Data

In [3]:
df=pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

### Check the balance of Data

In [6]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

## Split Data to Train and Test

In [7]:
features=df.iloc[:,:-1]
labels=df.iloc[:,-1]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features,labels, test_size=0.3,random_state=100, stratify=labels)

In [9]:
for dataset in [y_train, y_test]:
    print(round(len(dataset)/len(labels),2))
    print(dataset.value_counts())

0.7
0    199020
1       344
Name: Class, dtype: int64
0.3
0    85295
1      148
Name: Class, dtype: int64


## Normalized Data

In [10]:
Scaled=MinMaxScaler(feature_range = (0, 1))
#it makes more scence to fit on train data and then transform for all train, test, validation data.

In [11]:
X_train=Scaled.fit(X_train).transform(X_train)

In [12]:
X_test=Scaled.transform(X_test)

In [13]:
y_train.shape

(199364,)

# Create XGBOOST HYperparameter Toning

In [14]:
from xgboost import XGBClassifier
import xgboost

In [15]:
# defining the space for hyperparameter tuning

'''
hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.
'''

space={'max_depth': hp.quniform("max_depth", 6, 18, 2),
        'gamma': hp.uniform ('gamma', 1,4),
        'reg_alpha' : hp.quniform('reg_alpha', 40,60,5),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 4, 10, 2),
        'n_estimators': 180
    }


def hyperparameter_tuning(space):
    clf=xgboost.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'])
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    f1score = f1_score(y_test, pred>0.5)
    print ("SCORE:", f1score)
    #change the metric if you like
    return {'loss':-f1score, 'status': STATUS_OK }


# run the hyper paramter tuning
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print (best)

SCORE:                                                                                                                 
0.7471698113207548                                                                                                     
SCORE:                                                                                                                 
0.7500000000000001                                                                                                     
SCORE:                                                                                                                 
0.7601476014760147                                                                                                     
SCORE:                                                                                                                 
0.7565543071161048                                                                                                     
SCORE:                                  

SCORE:                                                                                                                 
0.7601476014760147                                                                                                     
SCORE:                                                                                                                 
0.7583643122676579                                                                                                     
SCORE:                                                                                                                 
0.7547169811320755                                                                                                     
SCORE:                                                                                                                 
0.7500000000000001                                                                                                     
SCORE:                                  

SCORE:                                                                                                                 
0.7452471482889733                                                                                                     
SCORE:                                                                                                                 
0.7356321839080461                                                                                                     
SCORE:                                                                                                                 
0.7593984962406016                                                                                                     
SCORE:                                                                                                                 
0.7500000000000001                                                                                                     
SCORE:                                  

In [16]:
# Model with best parameters 
clf = xgboost.XGBClassifier(colsample_bytree= 0.8491638389525318,
 gamma = 3.2889847866552087,
 max_depth=12,
 min_child_weight=4,
 reg_alpha = 45,
 reg_lambda = 0.725462201138424, 
#  eta= 0.8, nthread=-1, n_estimators=50, 
 seed=123)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85278    17]
 [   47   101]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.86      0.68      0.76       148

    accuracy                           1.00     85443
   macro avg       0.93      0.84      0.88     85443
weighted avg       1.00      1.00      1.00     85443




Adding scale_pos_weight hyperparameter since it is an imbalanced dataset:

In [17]:
# defining the space for hyperparameter tuning

'''
hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.
'''

space={'max_depth': hp.quniform("max_depth", 10, 20, 3),
        'gamma': hp.uniform ('gamma', 1,5),
        'reg_alpha' : hp.quniform('reg_alpha', 4,10,2),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
       'scale_pos_weight': sum(y_train==0)/ sum(y_train==1)
    }




def hyperparameter_tuning(space):
    clf=xgboost.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'], scale_pos_weight = space['scale_pos_weight'])
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    f1score = f1_score(y_test, pred>0.5)
    print ("SCORE:", f1score)
    #change the metric if you like
    return {'loss':-f1score, 'status': STATUS_OK }


# run the hyper paramter tuning
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print (best)

SCORE:                                                                                                                 
0.8                                                                                                                    
SCORE:                                                                                                                 
0.8322147651006713                                                                                                     
SCORE:                                                                                                                 
0.8                                                                                                                    
SCORE:                                                                                                                 
0.8250825082508251                                                                                                     
SCORE:                                  

SCORE:                                                                                                                 
0.836734693877551                                                                                                      
SCORE:                                                                                                                 
0.8310810810810809                                                                                                     
SCORE:                                                                                                                 
0.8395904436860068                                                                                                     
SCORE:                                                                                                                 
0.8445945945945946                                                                                                     
SCORE:                                  

SCORE:                                                                                                                 
0.8424657534246575                                                                                                     
SCORE:                                                                                                                 
0.8395904436860068                                                                                                     
SCORE:                                                                                                                 
0.8322147651006713                                                                                                     
SCORE:                                                                                                                 
0.8294314381270902                                                                                                     
SCORE:                                  

In [18]:
# Model with best parameters 
clf = xgboost.XGBClassifier(colsample_bytree= 0.888725180329427,
 gamma = 1.515970584279983,
 max_depth=21,
 min_child_weight=4,
 reg_alpha = 4,
 reg_lambda = 0.32726844547082934,
 scale_pos_weight= sum(y_train==0)/ sum(y_train==1), 
#  eta= 0.3, 
#  nthread=-1, 
#  n_estimators=180, 
 seed=123)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85270    25]
 [   25   123]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.83      0.83      0.83       148

    accuracy                           1.00     85443
   macro avg       0.92      0.92      0.92     85443
weighted avg       1.00      1.00      1.00     85443

